# Pré-traitement des données
Voici quelques étapes à suivre pour pré-traiter les données pour une utilisation avec K-means :

1. Intégration des données : Intégrez les données des différentes sources en une seule source de données. Si vous travaillez avec des données provenant de plusieurs sources, vous devez les intégrer en une seule source de données. Cela vous permettra de traiter les données comme une seule entité et de les analyser plus facilement.

1. Nettoyer les données : Assurez-vous que vos données sont propres et ne contiennent pas de valeurs manquantes, de doublons ou d'autres anomalies. Si nécessaire, supprimez ou remplacez les données manquantes.

2. Normaliser les données : Normalisez les données en les mettant à l'échelle de sorte que chaque variable ait une plage de valeurs comparable. La normalisation peut être effectuée en utilisant la méthode de la moyenne et de l'écart type, la méthode de la plage ou la méthode de la normalisation de l'amplitude.

3. Réduire la dimensionnalité : Si vous travaillez avec des données à haute dimensionnalité, utilisez des techniques de réduction de dimensionnalité telles que l'analyse en composantes principales (PCA) pour réduire la dimensionnalité de vos données et faciliter leur analyse.

4. Identifier les valeurs aberrantes : Les valeurs aberrantes peuvent fausser les résultats de K-means, il est donc important de les identifier et de les traiter correctement. Les valeurs aberrantes peuvent être supprimées ou remplacées par des valeurs plus appropriées.

5. Sélectionner les caractéristiques : Si vous travaillez avec des données qui contiennent de nombreuses caractéristiques, il peut être judicieux de sélectionner les caractéristiques les plus pertinentes pour votre analyse.

En résumé, le pré-traitement des données pour K-means comprend le nettoyage des données, la normalisation des données, la réduction de la dimensionnalité, l'identification et le traitement des valeurs aberrantes et la sélection des caractéristiques. En effectuant ces étapes, vous pouvez améliorer la qualité de vos données et obtenir des résultats plus significatifs à l'aide de K-means.

## Pipeline
La pipeline fait référence à un ensemble d'étapes ou de processus qui sont exécutés séquentiellement pour traiter les données d'entrée, extraire des caractéristiques, entraîner le modèle et prédire les résultats.

In [187]:
import pandas as pd
import numpy as np
import os
from copy import deepcopy

# Settings
pd.set_option('display.max_columns', None)

# Global variables
GENERAL_DATA_PATH = "./data/general_data.csv"
EMPLOYEE_SURVEY_DATA_PATH = "./data/employee_survey_data.csv"
MANAGER_SURVEY_DATA_PATH = "./data/manager_survey_data.csv"
IN_TIME_DATA_PATH = "./data/in_out_time/in_time.csv"
OUT_TIME_DATA_PATH = "./data/in_out_time/out_time.csv"

## Acquisition des données
Collecter des données brutes à partir de diverses sources, telles que des capteurs, des fichiers, des bases de données, etc.

In [188]:
def load_data(data_path):
    csv_path = os.path.join(data_path)
    return pd.read_csv(csv_path, sep=',')

def agregate_dataframes(list_df, on_column, how):
    df_master = list_df[0]
    for df in list_df[1:]:
        df_master = df_master.merge(df,
                       on = on_column, 
                       how = how)
    return df_master

# Load data
df_general = load_data(GENERAL_DATA_PATH)
df_employee_survey = load_data(EMPLOYEE_SURVEY_DATA_PATH)
df_manager_survey = load_data(MANAGER_SURVEY_DATA_PATH)
df_in_time = load_data(IN_TIME_DATA_PATH)
df_out_time = load_data(OUT_TIME_DATA_PATH)
# Merge dataframes
df_total = agregate_dataframes(
    [df_general, df_employee_survey, df_manager_survey],
    "EmployeeID",
    "outer"
)

# Display a sample of 5 rows
df_total.sample(5)

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
3262,35,No,Travel_Rarely,Research & Development,5,4,Medical,1,3263,Male,3,Manager,Married,16750,1.0,Y,16,8,0,3.0,6,3,0,2,1.0,4.0,2.0,2,3
2514,35,No,Travel_Rarely,Research & Development,8,1,Technical Degree,1,2515,Female,3,Manufacturing Director,Married,53460,0.0,Y,14,8,0,6.0,3,5,0,3,2.0,3.0,4.0,2,3
3051,31,No,Travel_Rarely,Research & Development,1,3,Life Sciences,1,3052,Male,4,Human Resources,Single,28670,0.0,Y,24,8,0,3.0,5,2,2,2,3.0,2.0,3.0,3,4
1975,34,No,Travel_Rarely,Research & Development,2,2,Life Sciences,1,1976,Male,1,Sales Representative,Married,64100,1.0,Y,13,8,1,1.0,5,1,0,0,3.0,3.0,3.0,2,3
634,19,Yes,Non-Travel,Sales,2,2,Other,1,635,Male,3,Healthcare Representative,Single,98240,1.0,Y,18,8,0,1.0,2,1,0,0,1.0,2.0,4.0,3,3


## Prétraitement des données
Nettoyer, normaliser et préparer les données pour l'analyse ultérieure.

### Nettoyage des données
Une étude préablable a permis de déterminer quels champs n'apportaient pas d'informations utiles à la prédiction. De même pour les champs non éthiques.

In [189]:
fields_not_useful = [
    "Over18",  # We have age
    "EducationField",  # We have Education
    "EmployeeCount",  # All at 1
    "StockOptionLevel",  # Not relevant
]
fields_not_ethical = [
    "Gender",
    "MaritalStatus",
]

# Datframe with only useful fields
df_useful = deepcopy(df_total)
df_useful.drop(
    fields_not_useful,
    axis=1,
    inplace=True
)
print("Shape useful:", df_useful.shape)
# Datframe with useful and ethical fields
df_ethical = deepcopy(df_total)
df_ethical.drop(
    fields_not_useful + fields_not_ethical,
    axis=1,
    inplace=True
)
print("Shape useful:", df_ethical.shape)

# Delete to free memory
del df_total
del fields_not_useful
del fields_not_ethical

Shape useful: (4410, 25)
Shape useful: (4410, 23)


### Normalisation des données

In [190]:
from sklearn.base import BaseEstimator, TransformerMixin

# List of columns to merge
columns_to_merge = [
    ("TotalWorkingYears", "Age"),
    ("YearsAtCompany", "Age"),
    ("NumCompaniesWorked", "Age"),
    ("YearsSinceLastPromotion", "Age"),
    ("YearsWithCurrManager", "Age"),
    ("TotalWorkingYears", "YearsAtCompany"),
    ("TotalWorkingYears", "YearsSinceLastPromotion"),
    ("TotalWorkingYears", "YearsWithCurrManager"),
]
column_ids_to_merge = []

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        global column_ids_to_merge
        new_columns = []
        print(column_ids_to_merge)
        for ctm in column_ids_to_merge:
            col_1 = X[:, ctm[0]]
            col_2 = X[:, ctm[1]]
            # index1 * index2
            new_columns.append(col_1 * col_2)
        return np.c_[X, *new_columns]

In [191]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedShuffleSplit

def split_train_test(data, test_ratio):
    """
    Split the data into a training set and a test set.
    """
    split = StratifiedShuffleSplit(n_splits=1, test_size=test_ratio, random_state=42)
    for train_index, test_index in split.split(data, data["Attrition"]):
        strat_train_set = data.loc[train_index]
        strat_test_set = data.loc[test_index]
    return strat_train_set, strat_test_set

def get_named_columns(columns: list[tuple[str]]=columns_to_merge) -> list[str]:
    """
    Return a list of columns names with the format "column1PerColumn2"
    """
    named_columns = []
    for ctm in columns:
        named_columns.append(ctm[0] + "Per" + ctm[1])
    return named_columns

def get_columns_ids(names: list[str], columns: list[tuple[str]]=columns_to_merge) -> list[tuple[int]]:
    """
    Return a list of columns ids from names (only if the column name is in "columns").
    """
    column_ids = []
    for c in columns:
        tmp = tuple()
        for i in c:
            tmp += (names.index(i),)
        column_ids.append(tmp)
    return column_ids

def fill_columns(m_list: list, nb_columns: int) -> list:
    """
    Fill the list to have the same length as nb_columns.
    """
    len_m_list = len(m_list)
    if len_m_list < nb_columns:
        for i in range(nb_columns - len_m_list):
            name = "Row_" + str(len_m_list + i)
            m_list.append(name)
    return m_list


def test(i, data1, data2):
    print("Test", i)
    print("Shape useful:", data1.shape)
    print("Shape ethical:", data2.shape)

# Split train and test sets
useful_train_set, useful_test_set = split_train_test(df_useful, 0.2)
ethical_train_set, ethical_test_set = split_train_test(df_ethical, 0.2)
test(2, useful_train_set, ethical_train_set)

# Create the pipeline for numerical attributes
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

# -- Useful --
useful_train_set_num = useful_train_set.select_dtypes(include=[np.number])
useful_num_attribs = list(useful_train_set_num)
column_ids_to_merge = get_columns_ids(useful_num_attribs)  # ids for useful set
useful_train_set_num_tr = num_pipeline.fit_transform(useful_train_set_num)
# -- Ethical --
ethical_train_set_num = ethical_train_set.select_dtypes(include=[np.number])
ethical_num_attribs = list(ethical_train_set_num)
column_ids_to_merge = get_columns_ids(ethical_num_attribs)  # ids for ethical set
ethical_train_set_num_tr = num_pipeline.fit_transform(ethical_train_set_num)

# List of categorical attributes
useful_cat_attribs = ["BusinessTravel", "Department", "JobRole", "Gender", "MaritalStatus",]
ethical_cat_attribs = ["BusinessTravel", "Department", "JobRole"]

useful_full_pipeline = ColumnTransformer([
    ("num", num_pipeline, useful_num_attribs),
    ("cat", OneHotEncoder(), useful_cat_attribs),
])
ethical_full_pipeline = ColumnTransformer([
    ("num", num_pipeline, ethical_num_attribs),
    ("cat", OneHotEncoder(), ethical_cat_attribs),
])

column_ids_to_merge = []
# -- Useful --
useful_train_set_prepared = useful_full_pipeline.fit_transform(useful_train_set)
# -- Ethical --
ethical_train_set_prepared = ethical_full_pipeline.fit_transform(ethical_train_set)
test(3, useful_train_set_prepared, ethical_train_set_prepared)

# Convert to dataframe to name columns
# -- Useful --
useful_columns = useful_num_attribs + get_named_columns() + useful_cat_attribs
useful_columns = fill_columns(useful_columns, useful_train_set_prepared.shape[1])
useful_train_set_prepared_df = pd.DataFrame(
    useful_train_set_prepared, columns=useful_columns
)
# -- Ethical --
ethical_columns = ethical_num_attribs + get_named_columns() + ethical_cat_attribs
ethical_columns = fill_columns(ethical_columns, ethical_train_set_prepared.shape[1])
ethical_train_set_prepared_df = pd.DataFrame(
    ethical_train_set_prepared, columns=ethical_columns
)

print("Useful set:")
print(ethical_train_set_prepared_df.sample(5).to_string())

# Delete to free memory
del columns_to_merge
del df_useful
del df_ethical
del useful_train_set
del useful_test_set
del ethical_train_set
del ethical_test_set
del useful_train_set_num
del useful_train_set_num_tr
del ethical_train_set_num
del ethical_train_set_num_tr
del useful_num_attribs
del ethical_num_attribs
del useful_cat_attribs
del ethical_cat_attribs
del useful_full_pipeline
del ethical_full_pipeline
del useful_train_set_prepared
del ethical_train_set_prepared

Test 2
Shape useful: (3528, 25)
Shape ethical: (3528, 23)
[(9, 0), (11, 0), (6, 0), (12, 0), (13, 0), (9, 11), (9, 12), (9, 13)]
[(9, 0), (11, 0), (6, 0), (12, 0), (13, 0), (9, 11), (9, 12), (9, 13)]
[]
[]
Test 3
Shape useful: (3528, 39)
Shape ethical: (3528, 34)
Useful set:
           Age  DistanceFromHome  Education  EmployeeID  JobLevel  MonthlyIncome  NumCompaniesWorked  PercentSalaryHike  StandardHours  TotalWorkingYears  TrainingTimesLastYear  YearsAtCompany  YearsSinceLastPromotion  YearsWithCurrManager  EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance  JobInvolvement  PerformanceRating  TotalWorkingYearsPerAge  YearsAtCompanyPerAge  NumCompaniesWorkedPerAge  YearsSinceLastPromotionPerAge  YearsWithCurrManagerPerAge  TotalWorkingYearsPerYearsAtCompany  TotalWorkingYearsPerYearsSinceLastPromotion  TotalWorkingYearsPerYearsWithCurrManager  BusinessTravel  Department  JobRole  Row_30  Row_31  Row_32  Row_33
56    1.233148          0.087413   -0.89898    1.336785 -0.969757 

## Extraction de caractéristiques
Extraire les caractéristiques pertinentes des données brutes pour les utiliser dans l'apprentissage automatique.

## Sélection des modèles
Sélectionner le modèle d'apprentissage automatique le plus approprié pour le problème spécifique que l'on cherche à résoudre.

## Entraînement du modèle
Entraîner le modèle sélectionné sur les données d'entraînement en utilisant des algorithmes d'apprentissage automatique appropriés.

## Évaluation du modèle
Evaluer les performances du modèle sur des données de test pour mesurer sa précision, sa fiabilité et sa robustesse.

## Mise en production du modèle
Intégrer le modèle entraîné dans une application en temps réel pour effectuer des prédictions sur de nouvelles données.